In [1]:
import numpy as np
import pandas as pd
import pyranges as pr
from pandas import DataFrame
import os
from collections import defaultdict

In [ ]:
#AF_events = pd.read_csv('localEvents/cerberus.events_AF_strict.ioe', sep = '\t')
#AL_events = pd.read_csv('localEvents/cerberus.events_AL_strict.ioe', sep = '\t')

In [2]:
metadata = pd.read_csv('swan_metadata.txt', sep = '\t')
metadata = metadata[['dataset','sample']]
d = defaultdict(list)
for a, b in metadata.values.tolist():
    d[b].append(a)

In [3]:
AF_psi = pd.read_csv('psi_by_suppa/cerberus_AF.psi', sep = '\t')
AL_psi = pd.read_csv('psi_by_suppa/cerberus_AL.psi', sep = '\t')

In [4]:
A3_psi = pd.read_csv('psi_by_suppa/cerberus_A3.psi', sep = '\t')
A5_psi = pd.read_csv('psi_by_suppa/cerberus_A5.psi', sep = '\t')
MX_psi = pd.read_csv('psi_by_suppa/cerberus_MX.psi', sep = '\t')
RI_psi = pd.read_csv('psi_by_suppa/cerberus_RI.psi', sep = '\t')
SE_psi = pd.read_csv('psi_by_suppa/cerberus_SE.psi', sep = '\t')

In [5]:
IC_psi = pd.concat([AF_psi, AL_psi, A3_psi, A5_psi, MX_psi, RI_psi, SE_psi])

In [ ]:
from collections import defaultdict

AF_dict = defaultdict(set)

for AF_event in AF_events['event_id']:
    gene, event = AF_event.split(';')[:2]
    chrom = event.split(':')[1]
    
    if event.endswith('-'):
        s1, e1 = event.split('-')[2].split(':')[:2]
        s2, e2 = event.split('-')[1].split(':')[:2]
        strand = '-'
    else:
        s1, e1 = event.split('-')[0].split(':')[2:]
        s2, e2 = event.split('-')[1].split(':')[1:]
        strand = '+'
          
    exon_1 = chrom + ':' + s1 + '-' + e1 + ':' + strand
    exon_2 = chrom + ':' + s2 + '-' + e2 + ':' + strand
    
    AF_dict[gene].add(exon_1)
    AF_dict[gene].add(exon_2)

In [6]:
cerberus_tss = pd.read_csv('psi_by_cerberus/tss_psi.tsv', sep = '\t', header = 0, low_memory = False)
cerberus_tes = pd.read_csv('psi_by_cerberus/tes_psi.tsv', sep = '\t', header = 0, low_memory = False)
cerberus_ic = pd.read_csv('psi_by_cerberus/ic_psi.tsv', sep = '\t', header = 0, low_memory = False)

In [7]:
def extract_genes_from_cerberus_across_sample(cerberus, sample, mapping, name, thres_1 = 0.25, thres_2 = 0.75):
    from collections import Counter
    df = cerberus.loc[cerberus['dataset'].isin(mapping[sample]), [name,'psi']].groupby([name]).mean()
    events = set(df[(df['psi'] >= thres_1) & (df['psi'] <= thres_2)].index)
    counter = Counter([event.split('_')[0] for event in events])
    return([event for event in counter.keys() if counter[event] >= 2])

In [8]:
def extract_genes_from_suppa_across_sample(psi, sample, mapping, threshold_1 = 0.25, threshold_2 = 0.75):    
    dataset = mapping[sample]
    df = psi[dataset]
    df = df[(df.mean(axis=1) >= threshold_1) & (df.mean(axis=1) <= threshold_2)]
    return(set([gene[0].split('.')[0] for gene in df.index.str.split(';')]))

In [40]:
def compare_cerberus_and_suppa_across_events(df, psi, name, mapping = d):
    
    path = '/'.join(['cerberus_suppa', name])
    if not os.path.exists(path):
        os.mkdir(path) 
    
    for sample in mapping.keys():
       
        geneList = set(df.loc[df['dataset'].isin(mapping[sample]), 'gid_stable'])
        gene_by_suppa = extract_genes_from_suppa_across_sample(psi, sample, mapping)
        gene_by_cerberus = extract_genes_from_cerberus_across_sample(df, sample, mapping, name)
        
        suppa = []; cerberus = []
        for gene in geneList:     
            
            suppa.append('TRUE') if gene in gene_by_suppa else suppa.append('FALSE') ## not detected by SUPPA: (1) psi value not in .25 - .75 (2) does not have a event
            cerberus.append('TRUE') if gene in gene_by_cerberus else cerberus.append('FALSE')
            
#            if gene in gene_by_cerberus:
#                cerberus.append('TRUE')
#            else:
#                cerberus.append('FALSE') if gene in gene_bg2 else cerberus.append(np.nan)            
#            lr.append(True) if gene in gene_by_lr else lr.append(False)
        res = DataFrame({'gid':list(geneList), 'sample':sample, 'suppa':suppa, 'cerberus':cerberus})
        # reformat
#        for i in range(df.shape[0]):
#            if(pd.isnull(df.iloc[i,3])):
 #               df.iloc[i,2] = np.nan
               
        res.to_csv(''.join([path, '/', sample, '.tsv']), sep = '\t', index = False, header = True)

In [42]:
compare_cerberus_and_suppa_across_events(df = cerberus_tss, psi = AF_psi, name = 'tss')
compare_cerberus_and_suppa_across_events(df = cerberus_tes, psi = AL_psi, name = 'tes')
compare_cerberus_and_suppa_across_events(df = cerberus_ic, psi = IC_psi, name = 'ic')

In [46]:
df = cerberus_tss.loc[cerberus_tss['dataset'].isin(d['a673']), ['tss','psi']].groupby(['tss']).mean()

In [57]:
events = set(df[(df['psi'] >= 0.1) & (df['psi'] <= 0.9)].index)
from collections import Counter
counter = Counter([event.split('_')[0] for event in events])

In [58]:
counter['ENSG00000124785']

2

In [56]:
'ENSG00000124785' in events 

False